# CREATE A CUSTOM MODEL

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import tensorflow as tf
import numpy as np

SEED = 345
tf.random.set_seed(SEED)  

cwd = os.getcwd()

## CREATE A GENERATORS

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        brightness_range=(0.5,1.),
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant', 
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)

## IMPORT TRAINING AND VALIDATION DATA

In [4]:
dataset_dir = os.path.join(cwd, 'newDataset')

bs = 5

img_h=256
img_w=256

num_classes= 20

classes = [ 'owl',    # 0
            'galaxy', # 1
            'lightning', # 2
            'wine-bottle', # 3
            't-shirt', # 4
            'waterfall', # 5
            'sword', # 6
            'school-bus', # 7
            'calculator', # 8
            'sheet-music', # 9
            'airplanes', # 10
            'lightbulb', # 11
            'skyscraper', # 12
            'mountain-bike', # 13
            'fireworks', # 14
            'computer-monitor', # 15
            'bear', # 16
            'grand-piano', # 17
            'kangaroo', # 18
            'laptop']       # 19


# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h,img_w),
                                               batch_size=bs, 
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               target_size=(img_h,img_w),
                                               batch_size=bs, 
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)  # targets are directly converted into one-hot vectors


class_indices = train_gen.class_indices
class_indices

Found 1244 images belonging to 20 classes.
Found 310 images belonging to 20 classes.


{'owl': 0,
 'galaxy': 1,
 'lightning': 2,
 'wine-bottle': 3,
 't-shirt': 4,
 'waterfall': 5,
 'sword': 6,
 'school-bus': 7,
 'calculator': 8,
 'sheet-music': 9,
 'airplanes': 10,
 'lightbulb': 11,
 'skyscraper': 12,
 'mountain-bike': 13,
 'fireworks': 14,
 'computer-monitor': 15,
 'bear': 16,
 'grand-piano': 17,
 'kangaroo': 18,
 'laptop': 19}

## TEST THE GENERATORS (SKIP THIS CODE)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import *

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                              output_types=(tf.float32,tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
num_row=10

fig, ax = plt.subplots(num_row, bs, figsize=(20, 20))
ax = ax.ravel()

iterator = iter(train_dataset)

for row in range(num_row):
    augmented_img, target = next(iterator)
    for col in range(0,5):
    #augmented_img = augmented_img[0]   # First element
    #augmented_img = augmented_img * 255  # denormalize
        _ = ax[bs*row+col].imshow(augmented_img[col]) 

Using TensorFlow backend.


## CREATE A MODEL AND INITIALIZE IT

In [55]:
from tensorflow.keras import layers
depth = 5
start_f = 8


model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(img_h, img_w, 3)))
for i in range(depth):
    model.add(layers.Conv2D(filters=start_f,
                                             kernel_size=(3, 3),
                                             strides=(1, 1), 
                                             padding='valid'))
   
    model.add(layers.ReLU())  # we can specify the activation function directly in Conv2D
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=0.3))
    start_f *= 2
    
model.add(layers.Flatten())
model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(units=num_classes, activation='softmax'))    

_ = tf.keras.utils.plot_model(model,'model_plot.png', show_shapes=True, show_layer_names=True,expand_nested=False)

In [44]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-2
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## SET CHECKPOINTS AND TRAINING THE MODEL

In [45]:
import os
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'),
                                                   monitor='val_acc',
                                                   save_weights_only=False,
                                                   save_best_only=True,
                                                   period=5) 
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

history = model.fit_generator(train_gen,
                             validation_data = valid_gen, 
                             epochs = 100)

Epoch 1/100
 48/249 [====>.........................] - ETA: 1:16 - loss: 3.3130 - accuracy: 0.0333

KeyboardInterrupt: 

## VISUALIZE THE RESULT OF PERFORMANCE

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## TEST THE MODEL AND SAVE THE RESULTS

In [ ]:
from datetime import datetime

def create_csv(results, results_dir='./output/'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
import os
from PIL import Image
import numpy as np

image_filenames = next(os.walk('./startDataset/test'))[2]                          
results = {}

for image_name in image_filenames:
    img = Image.open('./startDataset/test/'+image_name).convert('RGB')
    img = img.resize((img_w,img_h))
    img_array = np.array(img)
    img_array = img_array*1./255 #normalization
    img_array = np.expand_dims(img_array,0) #needed for fixed dim of input in the model
    output = model.predict(img_array)
    prediction = np.where(output == np.max(output))[1]  
    print('Image: '+str(image_name)+ ' Class: '+ list(class_indices.keys())[int(prediction)])
    results[image_name] = int(prediction)
    
create_csv(results)